## Streaming model explorer for Haystack 2.0

*notebook by Tilde Thurium:
 [Mastodon](https://tech.lgbt/@annthurium) || [Twitter](https://twitter.com/annthurium) || [LinkedIn](https://www.linkedin.com/in/annthurium/)*

*Problem*: there are so many LLMs these days! Which model is the best for my use case?

This notebook uses [Haystack 2.0](https://docs.haystack.deepset.ai/v2.0/docs/intro) to compare the results of sending the same prompt to several different models.

This is a very basic demo where you can only compare a few models that support streaming responses. I'd like to support more models in the future, so watch this space for updates.


### Models

Haystack's [OpenAIGenerator](https://docs.haystack.deepset.ai/v2.0/docs/openaigenerator) and [CohereGenerator](https://docs.haystack.deepset.ai/v2.0/docs/coheregenerator) support streaming out of the box.

The other models use the [HuggingFaceTGIGenerator](https://docs.haystack.deepset.ai/v2.0/docs/huggingfacetgigenerator) and have [hosted inference endpoints](https://huggingface.co/inference-api).

### Prerequisites

You need [HuggingFace](https://huggingface.co/docs/hub/security-tokens), [Cohere](https://docs.cohere.com/docs/connector-authentication), and [OpenAI](https://help.openai.com/en/articles/4936850-where-do-i-find-my-api-key) API keys. Save them as secrets in your Colab. Click on the key icon in the left menu or [see detailed instructions here](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75).

In [ ]:
!pip install -U haystack-ai cohere-haystack

In order for `userdata.get` to work, these keys need to be saved as secrets in your Colab. Click on the key icon in the left menu or [see detailed instructions here](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75).

In [8]:
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack_integrations.components.generators.cohere import CohereGenerator
from haystack.components.generators import HuggingFaceTGIGenerator
from google.colab import userdata

open_ai_generator = OpenAIGenerator(api_key=userdata.get('OPENAI_API_KEY'))

cohere_generator = CohereGenerator(api_key=userdata.get('COHERE_API_KEY'))

hf_generator = HuggingFaceTGIGenerator(model="mistralai/Mistral-7B-v0.1",token=userdata.get('HF_API_KEY'))

hf_generator.warm_up()

hf_generator_2 = HuggingFaceTGIGenerator(model="tiiuae/falcon-7b-instruct",token=userdata.get('HF_API_KEY'))

hf_generator_2.warm_up()

hf_generator_3 = HuggingFaceTGIGenerator(model="bigscience/bloom",token=userdata.get('HF_API_KEY'))

hf_generator_3.warm_up()

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [16]:
from haystack.components.generators import GPTGenerator
MODELS: list[GPTGenerator] = [open_ai_generator, cohere_generator, hf_generator, hf_generator_2, hf_generator_3]

The `AppendToken` dataclass formats the output so that the model name is printed, and the text follows in chunks of 5 tokens.

In [19]:
from dataclasses import dataclass
import ipywidgets as widgets

def output():...

@dataclass
class AppendToken:
  output: widgets.Output
  chunks = []
  chunk_size = 5

  def __call__(self, chunk):
      with self.output:
        text = getattr(chunk, 'content', '')
        self.chunks.append(text)
        if len(self.chunks) == self.chunk_size:
          output_string = ' '.join(self.chunks)
          self.output.append_display_data(output_string)
          self.chunks.clear()

def multiprompt(prompt, models=MODELS):
  outputs = [widgets.Output(layout={'border': '1px solid black'}) for _ in models]
  display(widgets.HBox(children=outputs))

  for i, model in enumerate(models):
    model_name = getattr(model, 'model', '')
    outputs[i].append_display_data(f'Model name: {model_name}')
    model.streaming_callback = AppendToken(outputs[i])
    model.run(prompt)


In [20]:
multiprompt("Tell me a cyberpunk story about a black cat.")

This was a very silly example prompt. If you found this demo useful, let me know the kinds of prompts you tested it with!

 [Mastodon](https://tech.lgbt/@annthurium) || [Twitter](https://twitter.com/annthurium) || [LinkedIn](https://www.linkedin.com/in/annthurium/)

Thanks for following along.